In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Idea

- Obtener la mayor cantidad de links de páginas de Mercado Libre por cada búsqueda. Por ejemplo, la búsqueda "guitarras Cort".
- Luego, guardamos esos links en una lista, la cual vamos a recorrer para realizar el webscraping en cada página y obtener los datos que buscamos.
- Por ultimo, creamos el *data frame* y lo exportamos con un archivo CSV (Uno por cada marca).

# Generador de links para el web scraping 

In [121]:
#marca de la guitarra
product = "guitarra gibson"
# cantidad de paginas en las que va a buscar
cant_pag = 6

# remplazar espacios en el nombre del producto
product_dash = product.replace(" ", "-")
product_encoded = product.replace(" ", "%20")

# url base
base_url = f"https://listado.mercadolibre.com.ar/{product_dash}"

# genero dinámicamente los URLs
urls = [f"{base_url}#D[A:{product_encoded}]"]  # Primer URL con codificación

# transformo la cantidad de paginas en su patrón
cant_pag_encoded = ((cant_pag-1) * 50) + 1

# agrego los links a la lista
for i in range(51, cant_pag_encoded, 50): 
    urls.append(f"{base_url}_Desde_{i}_NoIndex_True")

# data lists
all_raw_data = []
all_prices = []

for url in urls:
    print(url)

https://listado.mercadolibre.com.ar/guitarra-gibson#D[A:guitarra%20gibson]
https://listado.mercadolibre.com.ar/guitarra-gibson_Desde_51_NoIndex_True
https://listado.mercadolibre.com.ar/guitarra-gibson_Desde_101_NoIndex_True
https://listado.mercadolibre.com.ar/guitarra-gibson_Desde_151_NoIndex_True
https://listado.mercadolibre.com.ar/guitarra-gibson_Desde_201_NoIndex_True


# Web scraping de los titulos y precios

In [122]:
for url in urls:
    print(url)
    result = requests.get(url).text
    doc = BeautifulSoup(result, "html.parser")
    
    body = doc.body
    main = body.main
    maindiv = main.div.contents[2].next_sibling
    section = maindiv.contents[1]
    cards_list = section.ol

    # extraer datos de la página actual
    for tag in cards_list.contents:
        tagsize = len(tag.find_all(class_='andes-money-amount__fraction'))
        
        # controlamos que tenga la información necesaria 
        if tagsize > 0 and len(tag.find_all(class_="")) > 2:
            guitar_product = tag.find_all(class_="poly-component__title")[0].string
            price = tag.find_all(class_='andes-money-amount__fraction')[tagsize - 2].string
            all_raw_data.append(guitar_product)
            print(guitar_product)
            all_prices.append(price)

https://listado.mercadolibre.com.ar/guitarra-gibson#D[A:guitarra%20gibson]
Guitarra Eléctrica Gibson Modern Collection Les Paul Tribute de Caoba Satin Honeyburst Laca Nitrocelulosa Satinada con Diapasón de Palo de Rosa
Gibson Les Paul Standard 60 S Plain Top Ebony
Guitarra Electrica Gibson Les Paul Special Tribute Prm
Guitarra eléctrica Gibson Artist Collection Slash Les Paul Standard de caoba appetite burst laca de nitrocelulosa con diapasón de palo de rosa
Guitarra Gibson Les Paul Standard 60 con tapa de cerezo con tapa figurada
Pack X3 Souvenir Guitarra Gibson Sg Acdc (o Surtido A Elec)
Guitarra eléctrica Gibson Modern Collection SG Tribute de caoba 2018 vintage cherry satin laca nitrocelulosa satinada con diapasón de palo de rosa
Gibson Les Paul Standard 60 Figured Top Honey Amber
Guitarra Electrica Gibson Flying V 80 S Ebony
Guitarra eléctrica Gibson ES-335 Traditional de álamo/arce 2018 antique faded cherry brillante con diapasón de palo de rosa
Guitarra Gibson Es 335 Figured Ant

# Creación del DataFrame y exportación como archivo CSV

In [118]:
df = pd.DataFrame({
    "title" : all_raw_data,
    "price" : all_prices
})

In [119]:
df

,title,price
0,Guitarra Eléctrica Gibson Modern Collection Le...,4.363.959
1,Gibson Les Paul Standard 60 S Plain Top Ebony,7.809.110
2,Guitarra Electrica Gibson Les Paul Special Tri...,3.055.553
3,Guitarra eléctrica Gibson Artist Collection Sl...,10.606.152
4,Guitarra eléctrica Gibson Artist Collection Sl...,10.605.152
...,...,...
122,Guitarra Electrica Gibson Sg Tribute Prm,3.920.745
123,Pack X3 Souvenir Guitarra Gibson Les Paul (o S...,9.100
124,Guitarra Electrica Gibson Sg Tribute,3.409.343
125,Gibson Les Paul Standard 50s Faded Vintage Hon...,5.945.776


In [120]:
df.to_csv("guitar_gibson.csv",index=False)